In [1]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
import os
import pandas as pd
import numpy as np
import collections
from sklearn.base import TransformerMixin
import random
import pandas_profiling

     - 17.9 MB 22.5 MB/s 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.7/679.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.7 MB/s eta 0:00:00
  Created wheel for ydata-profiling: filename=ydata_profiling-0.0.dev0-py2.py3-none-any.whl size=399386 sha256=baeb3ab52b01456b2e9eec49aecf2cfa591ce1a0969d9a9fcb901a649ed8a061
  Stored in directory: /tmp/pip-ephem-wheel-cache-2jorivms/wheels/71/3c/c6/1dde962797c52d907d477fe1

/tmp/ipython-input-1656296338.py:8: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


In [2]:
import seaborn as sns
import time
import re
import os
import matplotlib.pyplot as plt

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
sns.set(style="ticks")
spark = SparkSession.builder.appName('ml-bank').getOrCreate()

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import pandas as pd
df_csv = spark.read.csv('/content/gdrive/MyDrive/bank.csv', sep=';', header = True, inferSchema =
True)
df_csv.printSchema()

Mounted at /content/gdrive
root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [5]:
spark_df=df_csv
spark_df.head(5)
spark_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- y: string (nullable = true)



In [6]:
spark_df.count()


4521

In [7]:
len(spark_df.columns), spark_df.columns

(17,
 ['age',
  'job',
  'marital',
  'education',
  'default',
  'balance',
  'housing',
  'loan',
  'contact',
  'day',
  'month',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'poutcome',
  'y'])

In [8]:
spark_df.describe().show()

+-------+------------------+-------+--------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+----+
|summary|               age|    job| marital|education|default|           balance|housing|loan| contact|               day|month|          duration|          campaign|             pdays|          previous|poutcome|   y|
+-------+------------------+-------+--------+---------+-------+------------------+-------+----+--------+------------------+-----+------------------+------------------+------------------+------------------+--------+----+
|  count|              4521|   4521|    4521|     4521|   4521|              4521|   4521|4521|    4521|              4521| 4521|              4521|              4521|              4521|              4521|    4521|4521|
|   mean| 41.17009511170095|   NULL|    NULL|     NULL|   NULL|1422.6578190665782|   NULL|NULL|    NULL|15.9152842291528

In [9]:
spark_df.select('balance','y').show(5)

+-------+---+
|balance|  y|
+-------+---+
|   1787| no|
|   4789| no|
|   1350| no|
|   1476| no|
|      0| no|
+-------+---+
only showing top 5 rows



In [10]:
spark_df.crosstab('y', 'marital').show()

+---------+--------+-------+------+
|y_marital|divorced|married|single|
+---------+--------+-------+------+
|       no|     451|   2520|  1029|
|      yes|      77|    277|   167|
+---------+--------+-------+------+



In [11]:
# sample sets
sample1 = spark_df.sample(False, 0.2, 42)
sample2 = spark_df.sample(False, 0.2, 43)
# train set
train = spark_df.sample(False, 0.8, 44)
train.withColumn('balance_new', train.balance /2.0)
train.withColumn('balance_new', train.balance /2.0).select('balance','balance_new').show(5)


+-------+-----------+
|balance|balance_new|
+-------+-----------+
|   1787|      893.5|
|   1350|      675.0|
|   1476|      738.0|
|      0|        0.0|
|    747|      373.5|
+-------+-----------+
only showing top 5 rows



In [ ]:
import os
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
train_with_id = spark_df.withColumn("ID", monotonically_increasing_id())
train_with_id = train_with_id.withColumn('ID2', train_with_id.ID)
train_with_id1 = train_with_id.drop('balance', "ID2")
train_with_id2 = train_with_id.select('balance', "ID2")
train_with_id2 = train_with_id2.withColumnRenamed("ID2", "ID")

In [ ]:
train_merged = train_with_id1.join(train_with_id2, on=['ID'], how='left_outer')
train_merged

DataFrame[ID: bigint, age: int, job: string, marital: string, education: string, default: string, housing: string, loan: string, contact: string, day: int, month: string, duration: int, campaign: int, pdays: int, previous: int, poutcome: string, y: string, balance: int]

In [ ]:
train_subsetted = spark_df.filter(spark_df.balance > 0.0)
pd.DataFrame(train_subsetted.head(5))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,35,management,single,tertiary,no,747,no,no,cellular,23,feb,141,2,176,3,failure,no


In [ ]:
import plotly.graph_objs as go
from plotly.offline import plot, iplot
import plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
print(__version__)


6.5.0


In [ ]:
# Importing libraries
import numpy as np
import pandas as pd

# plotly packages
import plotly
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
cf.go_offline()

df = pd.read_csv('https://raw.githubusercontent.com/mertcank1/BDA/refs/heads/main/bank.csv', sep=';')

import plotly.express as px

# compute counts per category
counts = df['y'].value_counts().reset_index()
counts.columns = ['y', 'count']

# bar plot
fig = px.bar(counts, x='y', y='count', color='y')
fig.show()


In [ ]:
df
import plotly.express as px
fig = px.scatter(
    df,
    x="balance",
    y="age",
    color="pdays",
    color_continuous_scale=[(0, "red"), (1, "darkred")]
)

fig.show()


In [ ]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,services,married,secondary,no,-333,yes,no,cellular,30,jul,329,5,-1,0,unknown,no
4517,57,self-employed,married,tertiary,yes,-3313,yes,yes,unknown,9,may,153,1,-1,0,unknown,no
4518,57,technician,married,secondary,no,295,no,no,cellular,19,aug,151,11,-1,0,unknown,no
4519,28,blue-collar,married,secondary,no,1137,no,no,cellular,6,feb,129,4,211,3,other,no


In [ ]:
import plotly.express as px
fig = px.box(df, y="age")
fig.show()
